In [1]:
import sys
import numba
import swifter
import timeit
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

__author__ = 'HK Dambanemuya'
__version__ = 'Python 2'

In [2]:
data = pd.read_csv("../Data/bid_fe_revised.csv")
data.sample(5)

,ListingKey,BidCreationDate,MemberKey,BidKey,BidAmount,ParticipationAmount,ListingStatus,Status,ListingCreationDate,ListingStartDate
7653301,4E7D34986923926656B7245,2010-10-26 14:52:23,79CE3381502119641216E82,F9653602953828428C31C61,25.00,25.00,Completed,Winning,2010-10-26 09:16:14,2010-10-26 14:41:27
5398429,EEC63428354068905FA5C6F,2008-07-30 14:37:00,D6743420240239666C086EE,D094353192999003069C75B,50.00,0.00,Withdrawn,Outbid,2008-07-30 14:34:45,2008-07-30 14:35:39
3849965,222234147959734849524A7,2008-02-27 06:25:48,F01033794807872492FF99F,B8A93518209130306459A91,50.00,50.00,Completed,Winning,2008-02-26 23:35:40,2008-02-26 23:47:20
3072843,967F3408158435156B7E544,2007-12-11 08:29:43,47A43366032490436748D50,0DE03511474230473E79CF2,190.00,0.00,Completed,Outbid,2007-12-06 14:16:57,2007-12-06 14:49:18
2863066,02ED340355421846781EC6D,2007-11-11 05:46:38,E8593398457375304186803,57E73509164921876C0339F,59.79,59.79,Completed,Winning,2007-11-04 10:08:24,2007-11-04 10:10:19


In [3]:
len(data)

9732656

In [4]:
hashmap = dict()
memberkeys = sorted(list(set(data.MemberKey)))[10000:20000]
len(memberkeys)

10000

In [5]:
start = timeit.default_timer()
for key in memberkeys:
    hashmap[key] = data.query("MemberKey == '{0}'".format(key))
stop = timeit.default_timer()
print "Hashmap generated in {0} seconds".format((stop-start)) 

Hashmap generated in 3164.6809115 seconds


In [6]:
data = data[data['MemberKey'].isin(memberkeys)]

In [7]:
len(data)

1484157

In [8]:
# data[["MemberKey", "ListingKey", "ListingStartDate", "BidKey", "BidCreationDate", "Status", "BidAmount", "ParticipationAmount"]].query("MemberKey=='{0}'".format(memberkeys[2])).sort_values("ListingStartDate")

## Lender Median Bid Amount

In [9]:
def lender_median_bid_amount(x):
    return np.median(hashmap[x["MemberKey"]].query("ListingStartDate < '{0}'".format(x["ListingStartDate"]))["BidAmount"])

start = timeit.default_timer()    
data["LenderMedianBidAmount"] = data[["MemberKey", "ListingStartDate"]].swifter.apply(lender_median_bid_amount, axis=1)
stop = timeit.default_timer()
print "Feature engineering completed in {0} seconds".format((stop-start)) 

Feature engineering completed in 3018.8430712 seconds


In [10]:
# data[["MemberKey", "ListingKey", "ListingStartDate", "BidKey", "BidCreationDate", "Status", "BidAmount", "ParticipationAmount", "LenderMedianBidAmount"]].query("MemberKey=='{0}'".format(memberkeys[2])).sort_values("ListingStartDate")

## Lender Mean Bid Amout

In [11]:
def lender_mean_bid_amount(x):
    return np.mean(hashmap[x["MemberKey"]].query("ListingStartDate < '{0}'".format(x["ListingStartDate"]))["BidAmount"])

start = timeit.default_timer()    
data["LenderMeanBidAmount"] = data[["MemberKey", "ListingStartDate"]].swifter.apply(lender_mean_bid_amount, axis=1)
stop = timeit.default_timer()
print "Feature engineering completed in {0} seconds".format((stop-start)) 

Feature engineering completed in 3066.3744641 seconds


In [12]:
# data[["MemberKey", "ListingKey", "ListingStartDate", "BidKey", "BidCreationDate", "Status", "BidAmount", "ParticipationAmount", "LenderMedianBidAmount", "LenderMeanBidAmount"]].query("MemberKey=='{0}'".format(memberkeys[2])).sort_values("ListingStartDate")

## Lender Max Bid Amount

In [13]:
def lender_max_bid_amount(x):
    return np.max(hashmap[x["MemberKey"]].query("ListingStartDate < '{0}'".format(x["ListingStartDate"]))["BidAmount"])

start = timeit.default_timer()    
data["LenderMaxBidAmount"] = data[["MemberKey", "ListingStartDate"]].swifter.apply(lender_max_bid_amount, axis=1)
stop = timeit.default_timer()
print "Feature engineering completed in {0} seconds".format((stop-start)) 

Feature engineering completed in 3018.0943099 seconds


In [14]:
# data[["MemberKey", "ListingKey", "ListingStartDate", "BidKey", "BidCreationDate", "Status", "BidAmount", "ParticipationAmount", "LenderMedianBidAmount", "LenderMeanBidAmount", "LenderMaxBidAmount"]].query("MemberKey=='{0}'".format(memberkeys[2])).sort_values("ListingStartDate")

## Lender Sum Bid Amount

In [15]:
def lender_sum_bid_amount(x):
    return np.sum(hashmap[x["MemberKey"]].query("ListingStartDate < '{0}'".format(x["ListingStartDate"]))["BidAmount"])

start = timeit.default_timer()    
data["LenderSumBidAmount"] = data[["MemberKey", "ListingStartDate"]].swifter.apply(lender_sum_bid_amount, axis=1)
stop = timeit.default_timer()
print "Feature engineering completed in {0} seconds".format((stop-start)) 

Feature engineering completed in 3022.5846485 seconds


In [16]:
data[["MemberKey", "ListingKey", "ListingStartDate", "BidKey", "BidCreationDate", "Status", "BidAmount", "ParticipationAmount", "LenderMedianBidAmount", "LenderMeanBidAmount", "LenderMaxBidAmount", "LenderSumBidAmount"]].query("MemberKey=='{0}'".format(memberkeys[2])).sort_values("ListingStartDate")

,MemberKey,ListingKey,ListingStartDate,BidKey,BidCreationDate,Status,BidAmount,ParticipationAmount,LenderMedianBidAmount,LenderMeanBidAmount,LenderMaxBidAmount,LenderSumBidAmount
3055437,27693406673501223ADCB6C,465634058437445298A8B5E,2007-11-29 10:30:53,BAE4351128578899452CDB5,2007-12-03 11:24:10,Outbid,500.0,0.0,NaN,NaN,NaN,0.0
3055439,27693406673501223ADCB6C,CB0534073651114089CA8DF,2007-12-02 12:22:24,45D4351051498459327AA4D,2007-12-11 12:14:29,Winning,50.0,50.0,500.0,500.000000,500.0,500.0
3055438,27693406673501223ADCB6C,744E340613588850873B15A,2007-12-02 22:37:12,53CB35117956109196B8BF0,2007-12-11 12:14:30,Winning,50.0,50.0,275.0,275.000000,500.0,550.0
3150582,27693406673501223ADCB6C,002B340592777918822C7D4,2007-12-04 08:29:19,86963511288790334E07DED,2007-12-11 12:14:31,Winning,50.0,50.0,50.0,200.000000,500.0,600.0
3343451,27693406673501223ADCB6C,4525340787894249506C91A,2007-12-04 11:45:46,FD7C3511275022248BFBFC2,2007-12-11 12:14:28,Winning,50.0,50.0,50.0,162.500000,500.0,650.0
3055440,27693406673501223ADCB6C,1B0A34079158318628B685F,2007-12-04 23:16:39,D7E735106651981517B67F8,2007-12-11 12:14:30,Winning,50.0,50.0,50.0,140.000000,500.0,700.0
3247523,27693406673501223ADCB6C,58F3340712108260398087C,2007-12-05 19:05:30,301C351196438054754B51F,2007-12-11 12:14:30,Winning,50.0,50.0,50.0,125.000000,500.0,750.0
3150581,27693406673501223ADCB6C,164B34076451096479FCEE3,2007-12-06 06:00:48,3D6F35120600466823BF908,2007-12-13 06:48:53,Winning,50.0,50.0,50.0,114.285714,500.0,800.0
3150583,27693406673501223ADCB6C,20A53406411728868168D71,2007-12-06 19:07:59,15AF3510193430405EC442B,2007-12-11 12:14:31,Winning,50.0,50.0,50.0,106.250000,500.0,850.0
3055441,27693406673501223ADCB6C,0E2434070840530686B28AB,2007-12-07 14:02:26,ABE23510933580993750626,2007-12-12 12:33:46,Outbid,50.0,0.0,50.0,100.000000,500.0,900.0


## Save Data

In [17]:
data.to_csv("../Data/lender_bidamount_fe/lender_bidamount_fe_20k.csv", index=False)